In [2]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

### Importing data

In [3]:
data = {'PostalCode':['2770000'],'Country':['Isla de Pascua'],'City':'Hanga Roa'}
df = pd.DataFrame.from_dict(data) 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head()

,PostalCode,Country,City
0,2770000,Isla de Pascua,Hanga Roa


In [4]:
df.shape

(1, 3)

### Getting coordinates 

In [5]:
address = 'Hanga Roa'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hanga Roa are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Hanga Roa are -27.1481172, -109.4273444.


In [6]:
df['latitude'] = latitude
df['longitude'] = longitude

In [7]:
df.head()

,PostalCode,Country,City,latitude,longitude
0,2770000,Isla de Pascua,Hanga Roa,-27.148117,-109.427344


### Exploring the capital of Easter island 

In [8]:
my_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, country, city in zip(df['latitude'], df['longitude'], df['Country'], df['City']):
    label = '{}, {}'.format(df['City'], df['Country'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(my_map)  
    
my_map

In [8]:
CLIENT_ID = '42GRWVXVEWIG1PDBQKXYVHTFUF01ZPLEWYZJXNHWROUTKDW2' 
CLIENT_SECRET = '3KYQYGXOY0HO1XL2TE4OKBLBSTZCC11AOF4OEPQKY4C04PET' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 42GRWVXVEWIG1PDBQKXYVHTFUF01ZPLEWYZJXNHWROUTKDW2
CLIENT_SECRET:3KYQYGXOY0HO1XL2TE4OKBLBSTZCC11AOF4OEPQKY4C04PET


In [9]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url 


'https://api.foursquare.com/v2/venues/explore?&client_id=42GRWVXVEWIG1PDBQKXYVHTFUF01ZPLEWYZJXNHWROUTKDW2&client_secret=3KYQYGXOY0HO1XL2TE4OKBLBSTZCC11AOF4OEPQKY4C04PET&v=20180605&ll=-27.1481172,-109.4273444&radius=500&limit=100'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e390da11835dd001b6acc46'},
 'response': {'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 41,
  'suggestedBounds': {'ne': {'lat': -27.143617195499996,
    'lng': -109.42229669103548},
   'sw': {'lat': -27.152617204500007, 'lng': -109.4323921089645}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51204b55e4b007500c353e60',
       'name': 'Makona',
       'location': {'address': 'Atamu tekena',
        'crossStreet': 'Te pito o te henua',
        'lat': -27.149318243108915,
        'lng': -109.428569317092,
        'labeledLatLngs': [{'label': 'display',
          'lat': -27.149318243108915,
          'lng': -109.428569317092}],
        'dist

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Getting all venues of Hanga Roa city 

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Makona,Restaurant,-27.149318,-109.428569
1,Mikafe,Café,-27.146846,-109.430587
2,Oheho Surf Cafe,Café,-27.147977,-109.430609
3,Café Ra'a,Breakfast Spot,-27.148552,-109.427876
4,Vai Te Mihi,Music Venue,-27.144356,-109.428494


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

41 venues were returned by Foursquare.


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
easter_island_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )


Hanga Roa


In [16]:
print(easter_island_venues.shape)
easter_island_venues.head()

(41, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hanga Roa,-27.148117,-109.427344,Makona,-27.149318,-109.428569,Restaurant
1,Hanga Roa,-27.148117,-109.427344,Mikafe,-27.146846,-109.430587,Café
2,Hanga Roa,-27.148117,-109.427344,Oheho Surf Cafe,-27.147977,-109.430609,Café
3,Hanga Roa,-27.148117,-109.427344,Café Ra'a,-27.148552,-109.427876,Breakfast Spot
4,Hanga Roa,-27.148117,-109.427344,Vai Te Mihi,-27.144356,-109.428494,Music Venue


In [40]:
easter_island_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Hanga Roa,40,40,40,40,40,40


In [20]:
# one hot encoding
easter_island_onehot = pd.get_dummies(easter_island_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
easter_island_onehot['Neighborhood'] = easter_island_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [easter_island_onehot.columns[-1]] + list(easter_island_onehot.columns[:-1])
easter_island_onehot = easter_island_onehot[fixed_columns]

easter_island_onehot.head()

,Neighborhood,Bar,Beach,Bed & Breakfast,Breakfast Spot,Café,Coffee Shop,Dance Studio,Diner,French Restaurant,Gift Shop,Harbor / Marina,History Museum,Hotel,Mountain,Music Venue,Nightclub,Park,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,South American Restaurant,Surf Spot
0,Hanga Roa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Hanga Roa,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hanga Roa,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hanga Roa,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hanga Roa,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
easter_island_onehot.shape

(40, 26)

In [22]:
easter_island_grouped = easter_island_onehot.groupby('Neighborhood').mean().reset_index()
easter_island_grouped

,Neighborhood,Bar,Beach,Bed & Breakfast,Breakfast Spot,Café,Coffee Shop,Dance Studio,Diner,French Restaurant,Gift Shop,Harbor / Marina,History Museum,Hotel,Mountain,Music Venue,Nightclub,Park,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,South American Restaurant,Surf Spot
0,Hanga Roa,0.025,0.05,0.025,0.025,0.075,0.025,0.025,0.025,0.025,0.05,0.025,0.025,0.125,0.025,0.025,0.025,0.025,0.025,0.025,0.125,0.025,0.1,0.025,0.025,0.025


In [23]:
easter_island_grouped.shape

(1, 26)

### Most common venues of Hanga Roa

In [24]:
num_top_venues = 5

for hood in easter_island_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = easter_island_grouped[easter_island_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Hanga Roa----
                venue  freq
0               Hotel  0.12
1          Restaurant  0.12
2  Seafood Restaurant  0.10
3                Café  0.08
4           Gift Shop  0.05




In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = easter_island_grouped['Neighborhood']

for ind in np.arange(easter_island_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(easter_island_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hanga Roa,Hotel,Restaurant,Seafood Restaurant,Café,Beach,Gift Shop,Harbor / Marina,Bed & Breakfast,Breakfast Spot,Coffee Shop


### All venues of Hanga Roa city

In [47]:
nearby_venues

,name,categories,lat,lng
0,Makona,Restaurant,-27.149318,-109.428569
1,Mikafe,Café,-27.146846,-109.430587
2,Feria Hanga Roa,Gift Shop,-27.149461,-109.429325
3,Oheho Surf Cafe,Café,-27.147977,-109.430609
4,Café Ra'a,Breakfast Spot,-27.148552,-109.427876
5,Vai Te Mihi,Music Venue,-27.144356,-109.428494
6,La Esquina,Pizza Place,-27.149125,-109.424828
7,Kuki Varua,Restaurant,-27.147013,-109.429900
8,Te Moana,Restaurant,-27.146321,-109.430565
9,Club Sandwich,Coffee Shop,-27.150368,-109.429667


### Sorting by count of venues in descending order

In [56]:
places = nearby_venues.groupby('categories')['name'].count().reset_index(name='count').sort_values(['count'], ascending=False)
places

,categories,count
12,Hotel,5
19,Restaurant,5
21,Seafood Restaurant,4
4,Café,3
9,Gift Shop,2
1,Beach,2
14,Music Venue,1
23,South American Restaurant,1
22,Snack Place,1
20,Sandwich Place,1


### Getting hotels of Hanga Roa

In [70]:
d = nearby_venues[nearby_venues['categories'] == 'Hotel']
d

,name,categories,lat,lng
26,Hotel Rapanui,Hotel,-27.149411,-109.426602
27,Hotel O'tai,Hotel,-27.147644,-109.428162
31,Hotel Casa Rapanui,Hotel,-27.149508,-109.425630
32,Hotel Tahatai,Hotel,-27.149500,-109.425455
39,HotelGomero,Hotel,-27.152431,-109.426658


In [71]:
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, venue in zip(d['lat'], d['lng'], d['name']):
    label = '{}, {}'.format(d['name'], d['categories'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

### Getting pharmacies of Hanga Roa

In [72]:
ph = nearby_venues[nearby_venues['categories'] == 'Pharmacy']
ph

,name,categories,lat,lng
25,Farmacias Cruz Verde,Pharmacy,-27.149947,-109.429249


In [75]:
pharmacy_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, venue in zip(ph['lat'], ph['lng'], ph['name']):
    label = '{}, {}'.format(ph['name'], ph['categories'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

### Conslusion 

According to Foursquare API, there are 5 hotels in Hanga Roa, Easter Island. Tourism is one of the most important 